In [3]:
from dataclasses import dataclass
import abc
from typing import Any, Union

import torch

@dataclass
class Rollout:
    x: int


In [1]:
from typing import Any, Dict, Optional  # noqa

import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as td
from optimrl.optimizer import LossFunction, RLOptimizer


### Env

In [2]:
env = gym.make("LunarLander-v2")
observation_dims = env.observation_space.shape[-1]
action_dims = env.action_space.n

In [3]:
action_dims

4

### Policy

In [4]:
def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer

In [5]:
class PPOCategoricalPolicy(nn.Module):
    def __init__(
        self,
        obs_dims,
        act_dims
    ):
        super().__init__()
        self.obs_dims = obs_dims
        self.act_dims = act_dims

        self.actor = nn.Sequential(
            layer_init(nn.Linear(self.obs_dims, 128)),
            nn.Tanh(),
            layer_init(nn.Linear(128, 128)),
            nn.Tanh(),
            layer_init(nn.Linear(128, self.act_dims))
        )

        self.critic = nn.Sequential(
            layer_init(nn.Linear(self.obs_dims, 128)),
            nn.ReLU(),
            layer_init(nn.Linear(128, 128)),
            nn.Tanh(),
            layer_init(nn.Linear(128, 1))
        )

    def forward(self, obs: torch.Tensor):
        logits = self.actor(obs)
        values = self.critic(obs)
        dist = td.Categorical(logits=logits)
        actions = dist.sample()
        log_probs = dist.log_prob(actions)
        return {"actions":actions, "values":values, "log_probs":log_probs, "dist":dist, "logits":logits}


In [6]:
initial, _ = env.reset()

In [7]:
initial.shape

(8,)

In [8]:
policy = PPOCategoricalPolicy(observation_dims, action_dims)

In [9]:
policy(torch.from_numpy(initial).unsqueeze(0))

{'actions': tensor([3]),
 'values': tensor([[-0.3831]], grad_fn=<AddmmBackward0>),
 'log_probs': tensor([-1.4580], grad_fn=<SqueezeBackward1>),
 'dist': Categorical(probs: torch.Size([1, 4]), logits: torch.Size([1, 4])),
 'logits': tensor([[0.2158, 0.0829, 0.1531, 0.0575]], grad_fn=<AddmmBackward0>)}

In [10]:
env.step(env.action_space.sample())

/Users/shyam/anaconda3/envs/py310/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


(array([ 0.00816231,  1.4143033 ,  0.4055499 ,  0.08813022, -0.01033811,
        -0.11043775,  0.        ,  0.        ], dtype=float32),
 0.06435684814009052,
 False,
 False,
 {})

### Rollout

In [27]:
envs = gym.vector.SyncVectorEnv(
    [lambda: gym.make("LunarLander-v2") for i in range(4)],
)

In [28]:
state, _ = envs.reset()

In [31]:
envs.single_action_space.shape

()

In [41]:
envs.num_envs

4

In [84]:
def ppo_rollout(envs, policy, num_steps: int, seed: int, device):
    num_envs = envs.num_envs
    obs = np.zeros((num_steps, num_envs) + envs.single_observation_space.shape)
    actions = np.zeros((num_steps, num_envs) + envs.single_action_space.shape)
    logprobs = np.zeros((num_steps, num_envs))
    rewards = np.zeros((num_steps, num_envs))
    dones = np.zeros((num_steps, num_envs))
    values = np.zeros((num_steps, num_envs))

    # TRY NOT TO MODIFY: start the game
    global_step = 0
    next_obs, _ = envs.reset(seed=seed)
    next_obs = next_obs
    next_done = np.zeros(num_envs)

    for step in range(num_steps):
        global_step += num_envs
        obs[step] = next_obs
        dones[step] = next_done

        # ALGO LOGIC: action logic
        out = policy(torch.Tensor(next_obs).to(device))
        values[step] = out["values"].squeeze().detach().cpu().numpy()
        action = out["actions"].detach().cpu().numpy()
        actions[step] = action
        logprobs[step] = out["log_probs"].squeeze().detach().cpu().numpy()

        # TRY NOT TO MODIFY: execute the game and log data.
        next_obs, reward, terminations, truncations, infos = envs.step(action)
        next_done = np.logical_or(terminations, truncations)
        rewards[step] = reward

    return obs, actions, logprobs, rewards, dones, values

In [85]:
envs = gym.vector.SyncVectorEnv(
    [lambda: gym.make("LunarLander-v2") for i in range(4)],
)

In [86]:
device = torch.device("cpu")
obs, actions, logprobs, rewards, dones, values = ppo_rollout(envs, policy, 10000, 1, device=device)

In [ ]:
obs = np.zeros((num_steps, num_envs) + envs.single_observation_space.shape)
actions = np.zeros((num_steps, num_envs) + envs.single_action_space.shape)
logprobs = np.zeros((num_steps, num_envs))
rewards = np.zeros((num_steps, num_envs))
dones = np.zeros((num_steps, num_envs))
values = np.zeros((num_steps, num_envs))

In [11]:
def ppo_rollout(
    policy,
    device
):
    SEED = None
    env = gym.make("LunarLander-v2")
    max_steps = 10000
    done = False
    observations, actions, rewards, logits, log_probs, values, terminals = (
        [],
        [],
        [],
        [],
        [],
        [],
        [],
    )
    count = 0
    observation, _ = env.reset(seed=SEED)
    with torch.no_grad():
        while not done:
            obs = torch.from_numpy(observation).unsqueeze(0).to(device)
            out = policy(obs)
            action = out["actions"].item()
            next_observation, reward, done, truncated, info = env.step(action)

            observations.append(observation)
            actions.append(action)
            rewards.append(reward)
            logits.append(out["logits"].squeeze().detach().cpu().numpy())
            log_probs.append(out["log_probs"].squeeze().detach().cpu().numpy())
            values.append(out["values"].squeeze().detach().cpu().numpy())
            terminals.append(done)

            observation = next_observation
            if count == max_steps:
                done = True
            count += 1
    env.close()

    return {
        "observations": np.array(observations),
        "actions": np.array(actions),
        "rewards": np.array(rewards),
        "log_probs": np.array(log_probs),
        "logits":np.array(logits),
        "values": np.array(values),
        "terminals": np.array(terminals),
    }



In [12]:
rollout = ppo_rollout(policy, torch.device("cpu"))

### PPO Loss

In [13]:
class PPOLossFunction(LossFunction):
    def __init__(
        self,
        vf_coef: float = 1.0,
        entropy_weight: float = 0.001,
        gamma: float = 0.99,
        clip_ratio: float = 0.2,
        norm_returns: bool = True,
        norm_advantages: bool = True
    ):
        self.vf_coef = vf_coef
        self.entropy_weight = entropy_weight
        self.gamma = gamma
        self.clip_ratio = clip_ratio
        self.norm_returns = norm_returns
        self.norm_advantages = norm_advantages

    def discount_rewards(self, rews: torch.Tensor, normalize=False) -> torch.Tensor:
        n = rews.shape[-1]
        rtgs = torch.zeros_like(rews)
        for i in reversed(range(n)):
            rtgs[:, i] = rews[:, i] + self.gamma * (rtgs[:, i + 1] if i + 1 < n else 0)
        if normalize:
            rtgs = (rtgs - rtgs.mean(dim=-1)) / rtgs.std(dim=-1)
        return rtgs

    def actor_loss(
        self,
        log_probs: torch.Tensor,
        old_logprobs: torch.Tensor,
        advantages: torch.Tensor,
    ) -> torch.Tensor:
        ratio = torch.exp(log_probs - old_logprobs.detach())
        assert tuple(ratio.size()) == tuple(advantages.size())
        surr1 = ratio * advantages
        surr2 = (
            torch.clamp(ratio, 1.0 - self.clip_ratio, 1.0 + self.clip_ratio)
            * advantages
        )
        loss_pi = -torch.min(surr1, surr2)
        return loss_pi

    def value_loss(self, values: torch.Tensor, returns: torch.Tensor) -> torch.Tensor:
        assert tuple(values.squeeze().size()) == tuple(returns.squeeze().size())
        return F.mse_loss(returns.squeeze(), values.squeeze())

    def __call__(
        self,
        log_probs: torch.Tensor,
        dist,
        old_log_probs: torch.Tensor,
        rewards: torch.Tensor,
        values: torch.Tensor,
        terminals: Optional[torch.Tensor] = None,
    ):
        returns = self.discount_rewards(rewards, normalize=self.norm_returns)
        advantages = returns - values
        if self.norm_advantages:
            advantages = (advantages - advantages.mean(dim=-1)) / advantages.std(dim=-1)
        actor_loss = self.actor_loss(log_probs, old_log_probs, advantages).mean()
        value_loss = self.value_loss(values, returns).mean()
        entropy_loss = dist.entropy().sum(1)
        loss = (
            actor_loss + self.vf_coef * value_loss - self.entropy_weight * entropy_loss
        )
        return loss.mean()


In [14]:
env = gym.make("LunarLander-v2")

In [15]:
loss_fn = PPOLossFunction(entropy_weight=0.01, norm_advantages=False)

In [16]:
def prepare_inputs(
    rollouts,
    device
):
    observations = torch.stack([torch.from_numpy(r["observations"]) for r in rollouts]).to(device)
    actions = torch.stack([torch.from_numpy(r["actions"]) for r in rollouts]).to(device)
    rewards = torch.stack([torch.from_numpy(r["rewards"]) for r in rollouts]).to(device)
    terminals = torch.stack([torch.from_numpy(r["terminals"]) for r in rollouts]).to(device)
    log_probs = torch.stack([torch.from_numpy(r["log_probs"]) for r in rollouts]).to(device)
    logits = torch.stack([torch.from_numpy(r["logits"]) for r in rollouts]).to(device)
    values = torch.stack([torch.from_numpy(r["values"]) for r in rollouts]).to(device)
    dist = td.Categorical(logits=logits)
    return {
        "observations":observations,
        "actions":actions,
        "rewards":rewards,
        "terminals":terminals,
        "log_probs":log_probs,
        "logits":logits,
        "dist":dist,
        "values":values,
    }


In [17]:
device = torch.device("cpu")

In [18]:
prepared_inputs = prepare_inputs([rollout], device=device)

In [19]:
prepared_inputs["log_probs"].shape

torch.Size([1, 87])

In [20]:
out = policy(prepared_inputs["observations"].squeeze())

In [21]:
out["log_probs"].unsqueeze(0).shape

torch.Size([1, 87])

In [22]:
loss = loss_fn(
    log_probs=out["log_probs"].unsqueeze(0),
    dist=prepared_inputs["dist"],
    old_log_probs=prepared_inputs["log_probs"],
    rewards=prepared_inputs["rewards"],
    values=prepared_inputs["values"],
    terminals=prepared_inputs["terminals"]
)


### Optimize step

In [23]:
import torch.optim as optim

class PPOOptimizer:
    def __init__(
        self,
        policy,
        loss_fn,
        pi_lr: float = 0.0005,
        n_updates: int = 4
    ):
        self.policy = policy
        self.loss_fn = loss_fn
        self.pi_lr = pi_lr
        self.n_updates = n_updates
        self.optimizer = optim.Adam(self.policy.parameters(), lr=self.pi_lr)

    def update(
        self,
        rollouts,
        device,
    ):
        prepared_inputs = prepare_inputs(rollouts, device=device)
        for i in range(self.n_updates):
            self.optimizer.zero_grad()
            out = self.policy(prepared_inputs["observations"].squeeze())
            loss = self.loss_fn(
                log_probs=out["log_probs"].unsqueeze(0),
                dist=prepared_inputs["dist"],
                old_log_probs=prepared_inputs["log_probs"],
                rewards=prepared_inputs["rewards"],
                values=prepared_inputs["values"],
                terminals=prepared_inputs["terminals"]
            )
            loss.backward()
            self.optimizer.step()
        return loss.item()


In [24]:
loss_fn = PPOLossFunction(entropy_weight=0.01, norm_advantages=False)
optimizer = PPOOptimizer(
    policy=policy,
    loss_fn=loss_fn,
    pi_lr=0.002,
    n_updates=4
)

In [25]:
from tqdm import tqdm

bar = tqdm(np.arange(50000))
device = torch.device("cpu")

rewards = []
for i in bar:
    with torch.no_grad():
        rollout = ppo_rollout(policy, device)
    sum_reward = np.sum(rollout["rewards"])
    loss = optimizer.update([rollout], device)
    rewards.append(sum_reward)
    # bar.set_description(f"Loss: {loss}, Sum reward: {np.mean(rewards[-20:])}")
    if i % 500 == 0:
        print(f"Loss: {loss}, Sum reward: {np.mean(rewards[-25:])}")

  0%|                                       | 0/50000 [00:00<?, ?it/s]/Users/shyam/anaconda3/envs/py310/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
  0%|                               | 9/50000 [00:00<20:05, 41.46it/s]

Loss: -0.9123230576515198, Sum reward: -121.96433337664078


  1%|▎                            | 505/50000 [00:12<21:42, 38.00it/s]

Loss: 0.021598339080810547, Sum reward: -204.8033545896234


  2%|▌                           | 1008/50000 [01:16<20:51, 39.13it/s]

...

 98%|████████████████████████▌| 49008/50000 [1:50:44<12:59,  1.27it/s]

Loss: -0.5504346489906311, Sum reward: -209.754436534842


 99%|████████████████████████▊| 49509/50000 [1:51:33<00:11, 41.42it/s]

Loss: -0.07728588581085205, Sum reward: -142.20115018727003


100%|█████████████████████████| 50000/50000 [1:51:45<00:00,  7.46it/s]
